In [1]:
## Imports
import pandas as pd
import json
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn import preprocessing


In [2]:
all = pd.read_pickle("../climate_classifier/scored_data/votes_data_all_expanded_politician.pkl")

In [33]:
all = all.reset_index(drop=True)
all["year"] = all["date"].astype("str").str[:4]
df = all[all["year"].astype("int")>=2012]

In [34]:
df[["For", "Absent", "Neither", "Against"]] = pd.get_dummies(df["vote"])
df.head(2)

C:\Users\COBOD\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,party,politician,vote,description,date,vote_id,y_pred,vote_id2,year,For,Absent,Neither,Against
0,Siumut (SIU),Aki-Matilda Høegh-Dam,Fraværende,Forslaget går ud på at lovfæste klippekortordn...,2022-03-15,"Afstemning nr. 320, 2021-22",non-climate,2821,2022,0,1,0,0
1,Dansk Folkeparti (DF),Alex Ahrendtsen,For,Forslaget går ud på at lovfæste klippekortordn...,2022-03-15,"Afstemning nr. 320, 2021-22",non-climate,2821,2022,1,0,0,0


In [35]:
df = df[["politician", "party", "For", "Against", "Neither", "Absent", "description", "vote_id2", "date", "y_pred"]]
df.sample(5)

,politician,party,For,Against,Neither,Absent,description,vote_id2,date,y_pred
279271,Ellen Trane Nørby,Venstre (V),1,0,0,0,Med lovforslaget afskaffes de særlige gyldighe...,4555,2017-06-08,non-climate
509061,John Dyrby Paulsen,Socialdemokratiet (S),0,1,0,0,Med beslutningsforslaget ønskedes det at tage ...,4581,2013-06-04,non-climate
135091,Orla Hav,Socialdemokratiet (S),1,0,0,0,"Det foreslås, at tilpasse reglerne for beskatn...",1440,2020-01-31,non-climate
91348,Jan Johansen,Socialdemokratiet (S),0,0,0,1,"Folketinget konstaterer, at langt de fleste ud...",2308,2021-01-22,non-climate
285733,Jacob Jensen,Venstre (V),0,0,0,1,Med lovforslaget forlænges det såkaldte økonom...,4540,2017-06-08,non-climate


### General description

In [31]:
df.nunique()

politician      408
party            19
For               2
Against           2
Neither           2
Absent            2
description    3296
vote_id2       3296
date            725
dtype: int64

In [7]:
#pd.DataFrame(df["politician"].unique()).to_csv("sanity_check_pol_votes.csv")

In [9]:
climate = df[df["y_pred"]=="climate"]
non_climate = df[df["y_pred"]=="non-climate"]
print(len(climate))
print(len(non_climate))
print(len(df))
climate.nunique()

42965
547101
590066


party           19
politician     398
vote             4
description    240
date           151
vote_id        240
y_pred           1
vote_id2       240
year            11
dtype: int64

In [10]:
non_climate.nunique()

party            19
politician      406
vote              4
description    3056
date            688
vote_id        3056
y_pred            1
vote_id2       3056
year             11
dtype: int64

#### Distributions of vote-outcomes

In [39]:
parties_selection = ["Socialdemokratiet (S)", "Venstre (V)", "Socialistisk Folkeparti (SF)", "Det Konservative Folkeparti (KF)", "Dansk Folkeparti (DF)", "Enhedslisten (EL)"
                    ,"Det Radikale Venstre (RV)", "Alternativet (ALT)", "Liberal Alliance (LA)"]
print(len(df))
df_selection = df[df["party"].isin(parties_selection)]
df_selection
print(len(df_selection))
df_selection

590066
567350


,politician,party,For,Against,Neither,Absent,description,vote_id2,date,y_pred
1,Alex Ahrendtsen,Dansk Folkeparti (DF),1,0,0,0,Forslaget går ud på at lovfæste klippekortordn...,2821,2022-03-15,non-climate
2,Alex Vanopslagh,Liberal Alliance (LA),0,0,0,1,Forslaget går ud på at lovfæste klippekortordn...,2821,2022-03-15,non-climate
3,Anders Kronborg,Socialdemokratiet (S),0,1,0,0,Forslaget går ud på at lovfæste klippekortordn...,2821,2022-03-15,non-climate
4,Andreas Steenberg,Det Radikale Venstre (RV),0,1,0,0,Forslaget går ud på at lovfæste klippekortordn...,2821,2022-03-15,non-climate
5,Ane Halsboe-Jørgensen,Socialdemokratiet (S),0,0,0,1,Forslaget går ud på at lovfæste klippekortordn...,2821,2022-03-15,non-climate
...,...,...,...,...,...,...,...,...,...,...
590061,Ulla Tørnæs,Venstre (V),1,0,0,0,Loven skal gøre beskæftigelsesindsatsen mere i...,5257,2012-01-15,non-climate
590062,Villum Christensen,Liberal Alliance (LA),0,0,1,0,Loven skal gøre beskæftigelsesindsatsen mere i...,5257,2012-01-15,non-climate
590063,Villy Søvndal,Socialistisk Folkeparti (SF),0,0,0,1,Loven skal gøre beskæftigelsesindsatsen mere i...,5257,2012-01-15,non-climate
590064,Zenia Stampe,Det Radikale Venstre (RV),1,0,0,0,Loven skal gøre beskæftigelsesindsatsen mere i...,5257,2012-01-15,non-climate


In [42]:
df_agree = df_selection.groupby(["party","y_pred"]).sum()[["For", "Against", "Neither", "Absent"]]
df_agree.head(4)

For  Against  Neither   Absent
party                 y_pred                                         
Alternativet (ALT)    climate        357.0    191.0     21.0    358.0
                      non-climate   5193.0   2792.0    243.0   5521.0
Dansk Folkeparti (DF) climate       2392.0    982.0    135.0   2428.0
                      non-climate  35105.0  11782.0   1338.0  32763.0

In [44]:
df_agree["Total"] = df_agree["For"]+df_agree["Against"]+df_agree["Neither"]
df_agree["Agree"] = df_agree["For"]/df_agree["Total"]
df_agree["Indifferent"] = df_agree["Neither"]/df_agree["Total"]
df_agree["Disagree"] = df_agree["Against"]/df_agree["Total"]
df_agree["Absence"] = df_agree["Absent"]/df_agree["Total"]
df_agree.head(4)

For  Against  Neither   Absent  \
party                 y_pred                                            
Alternativet (ALT)    climate        357.0    191.0     21.0    358.0   
                      non-climate   5193.0   2792.0    243.0   5521.0   
Dansk Folkeparti (DF) climate       2392.0    982.0    135.0   2428.0   
                      non-climate  35105.0  11782.0   1338.0  32763.0   

                                     Total     Agree  Indifferent  Disagree  \
party                 y_pred                                                  
Alternativet (ALT)    climate        569.0  0.627417     0.036907  0.335677   
                      non-climate   8228.0  0.631138     0.029533  0.339329   
Dansk Folkeparti (DF) climate       3509.0  0.681676     0.038472  0.279852   
                      non-climate  48225.0  0.727942     0.027745  0.244313   

                                    Absence  
party                 y_pred                 
Alternativet (ALT)    climate      0.629174  
                      non-climate  0.671001  
Dansk Folkeparti (DF) climate      0.691935  
                      non-climate  0.679378

In [47]:
df_agree[["Agree", "Indifferent", "Disagree", "Absence"]].describe()

,Agree,Indifferent,Disagree,Absence
count,18.000000,18.000000,18.000000,18.000000
mean,0.691267,0.016849,0.291884,0.665627
std,0.059823,0.010922,0.055903,0.028820
min,0.571429,0.005116,0.187788,0.609773
25%,0.673208,0.008496,0.278580,0.640848
50%,0.694417,0.012783,0.294824,0.669398
75%,0.702420,0.026309,0.309968,0.684954
max,0.803571,0.038472,0.417582,0.714538


In [69]:
df_agree_all = df_agree[["For", "Against", "Neither", "Absent"]]
df_agree_all = df_agree_all.groupby("y_pred").sum()
df_agree_all = df_agree_all.append(pd.DataFrame(df_agree_all.sum(axis=0)).transpose())
df_agree_all.head(4)


,For,Against,Neither,Absent
climate,17778.0,6483.0,337.0,16527.0
non-climate,217571.0,93784.0,5073.0,209797.0
0,235349.0,100267.0,5410.0,226324.0


In [70]:
df_agree_all["Total"] = df_agree_all["For"]+df_agree_all["Against"]+df_agree_all["Neither"]
df_agree_all["Agree"] = df_agree_all["For"]/df_agree_all["Total"]
df_agree_all["Indifferent"] = df_agree_all["Neither"]/df_agree_all["Total"]
df_agree_all["Disagree"] = df_agree_all["Against"]/df_agree_all["Total"]
df_agree_all["Absence"] = df_agree_all["Absent"]/df_agree_all["Total"]
df_agree_all.head(4)

,For,Against,Neither,Absent,Total,Agree,Indifferent,Disagree,Absence
climate,17778.0,6483.0,337.0,16527.0,24598.0,0.722742,0.013700,0.263558,0.671884
non-climate,217571.0,93784.0,5073.0,209797.0,316428.0,0.687585,0.016032,0.296383,0.663017
0,235349.0,100267.0,5410.0,226324.0,341026.0,0.690120,0.015864,0.294016,0.663656
